In [1]:
import pandas as pd

In [2]:
files_dire = '../data/raw/'

In [3]:
df = pd.read_csv(f'{files_dire}/melbourne_past_listings.csv')
# merge the past listing with the geocode
# this should only left with properties that have been listed in 2021-2022
geo_df = pd.read_csv(f'{files_dire}/geo.csv').iloc[: , 1:]
df = df.merge(geo_df, on='address')
df.head()

,year,month,bed,bath,car,land_raw,type,address,suburb,code,rent_raw,url,loc_address,lat,lon
0,2021,January,3.0,1.0,1.0,NaN,Rental_residential,"1/31 DANDENONG ROAD EAST, FRANKSTON",Frankston,3199,$330,https://www.oldlistings.com.au/real-estate/VIC...,"Dandenong Road East, Frankston, Melbourne, Cit...",-38.129890,145.132153
1,2012,February,3.0,1.0,1.0,NaN,Rental_residential,"1/31 DANDENONG ROAD EAST, FRANKSTON",Frankston,3199,$320 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Dandenong Road East, Frankston, Melbourne, Cit...",-38.129890,145.132153
2,2012,January,3.0,1.0,1.0,NaN,Rental_residential,"1/31 DANDENONG ROAD EAST, FRANKSTON",Frankston,3199,$340 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Dandenong Road East, Frankston, Melbourne, Cit...",-38.129890,145.132153
3,2021,January,3.0,1.0,2.0,NaN,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
4,2021,January,3.0,1.0,2.0,NaN,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494


In [4]:
# now review how many NaN exist in each column
df.count()

year           1444798
month          1444798
bed            1444783
bath           1444778
car            1334452
land_raw         84021
type           1181668
address        1444798
suburb         1444798
code           1444798
rent_raw       1444788
url            1444798
loc_address    1444798
lat            1444798
lon            1444798
dtype: int64

In [5]:
# we also remove the column of the size of the land as only very small number of rows have records
df = df.drop(['land_raw'], axis = 1)

In [6]:
# as the research goal is mainly residential properties (for people to live in)
# hence we filter out all properties that have zero bedroom or bathroom
temp = ['bed', 'bath', 'car']
df[temp] = df[temp].fillna(0)
df[temp] = df[temp].astype('int64')
df = df[~(df[temp] == 0).any(axis=1)]
# print out the number of entries left
len(df)

1334440

In [7]:
# from the previous output of NaN count, the types of property seems broken
df['type'].value_counts()

House                 472388
Unit/apmt             240217
Rental_residential    176164
Townhouse             109990
AvailableNow           59740
Unit                   17719
Apartment               5689
Available               3738
Villa                   2032
AvailableDate           1891
Studio                  1737
Flat                     835
Sales_residential         22
Rural                     10
ForSale                    5
Villa,House                3
Terrace                    3
Duplex                     1
Acreage/semi-rural         1
Name: type, dtype: int64

In [8]:
# some of the types has a really low count hence these are removed
temp = ['Duplex', 'Acreage/semi-rural', 'Terrace', 'Villa,House', 'ForSale', 'Rural', 'Sales_residential', 'AvailableDate', 'Villa', 'AvailableNow', 'Rental_residential', 'Available']

In [9]:
df = df[~df['type'].isin(temp)]

In [10]:
# we want to combine categories in to three: house; Apartment / Unit / Flat; townhouse; studio
df['type'].value_counts()

House        472388
Unit/apmt    240217
Townhouse    109990
Unit          17719
Apartment      5689
Studio         1737
Flat            835
Name: type, dtype: int64

In [11]:
# 'AUF' stands for 'Apartment / Unit / Flat'
df['type'] = df['type'].replace(['Unit/apmt', 'Apartment', 'Flat', 'Unit'], 'AUF')
df['type'].value_counts()

House        472388
AUF          264460
Townhouse    109990
Studio         1737
Name: type, dtype: int64

In [12]:
# preliminary processing on rent
# remove the dollar sign as well as comma
df['rent_raw'] = df['rent_raw'].str.replace(',', '')
df['rent_raw'] = df['rent_raw'].str.replace('$', '')
df.head()

/var/folders/y5/myhyydfd6h9dlwv1qmqtqhmr0000gn/T/ipykernel_21517/3517771870.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rent_raw'] = df['rent_raw'].str.replace('$', '')


,year,month,bed,bath,car,type,address,suburb,code,rent_raw,url,loc_address,lat,lon
3,2021,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
4,2021,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
5,2020,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
6,2020,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
7,2017,December,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395 pw,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494


In [13]:
# split the rent to a column of number and another for description
df['rent_raw'] = df['rent_raw'].str.lower()
df['rent_disc'] = df['rent_raw'].str.extract('(\D+)')
df['rent_raw'] = df['rent_raw'].str.extract('(\d+)')

In [14]:
# deduplicate entries again as some entries have slight difference such as 'per week' and 'pw'
deduplicate_subset = ['year', 'month', 'bed', 'bath', 'car', 'type', 'address', 'suburb', 'code', 'rent_raw']

In [15]:
df = df.drop_duplicates(subset=deduplicate_subset, keep=False)
# after deduplication, only properties listed more than once are retained
df = df[df.groupby('address').address.transform('count') > 1]

In [16]:
# remove all punctuation and space for further convinience
df['rent_disc'] = df['rent_disc'].str.replace(r'[^\w\s]+', '')
df['rent_disc'] = df['rent_disc'].str.replace(' ', '')

/var/folders/y5/myhyydfd6h9dlwv1qmqtqhmr0000gn/T/ipykernel_21517/3543244544.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['rent_disc'] = df['rent_disc'].str.replace(r'[^\w\s]+', '')


In [17]:
df['rent_disc'].value_counts()

perweek           177155
pw                106454
weekly             31928
                   23731
wk                  3354
                   ...  
forauction             1
priceonenquiry         1
millionweekly          1
justlisted             1
opentooffers           1
Name: rent_disc, Length: 84, dtype: int64

In [18]:
# sort rent into three types
week = ['perweek', 'pw', 'weekly', 'wk', 'week', 'perweekgst']
month = ['permonth', 'monthly', 'pcm', 'pm', 'month']
year = ['pa', 'perannum', 'annually']

In [19]:
df['rent_disc'] = df['rent_disc'].replace(week, 7)
df['rent_disc'] = df['rent_disc'].replace(month, 30)
df['rent_disc'] = df['rent_disc'].replace(year, 365)

In [20]:
# remove entries without proper rent descriptions
allowed_vals = [7, 30, 365]
df = df[df['rent_disc'].isin(allowed_vals)]
df['rent_disc'] = df['rent_disc'].astype('int')
df = df[~df['rent_raw'].isnull()]
df = df[df['rent_raw'].str.isnumeric()]
df['rent_raw'] = df['rent_raw'].astype('int')

In [21]:
# convert all rent into rent per week for consistency
df['weekly_rent'] =  df['rent_raw'] / df['rent_disc'] * 7
df = df.drop(['rent_disc', 'rent_raw'], axis=1)

In [22]:
# as all listing does not have a specific date
# all dates are assumed to be the first date of each month
# df['day'] = 1
df['month'] = pd.to_datetime(df.month, format='%B').dt.month
df['list_date'] = pd.to_datetime(df[['year', 'month']].assign(DAY=1))
df = df.drop(['year', 'month'], axis=1)

In [23]:
df.head()

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date
9,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0,2016-12-01
10,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2016-12-01
11,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2016-01-01
12,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2015-11-01
13,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,420.0,2021-01-01


In [24]:
# check once again to make sure there is no duplicate and less than 2 record
df = df[df.groupby('address').address.transform('count') > 1]
len(df)

303640

In [25]:
# check the number of unique properties in future investigation
len(list(set(df['address'].tolist())))

82556

In [27]:
df.to_csv('../data/curated/processed_listing.csv')

In [28]:
df.iloc[:1000, :].to_csv('../data/curated/SAMPLE_processed_listing.csv')